In [1]:
!pip install git+https://github.com/openai/whisper.git
!pip install jiwer


  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-unkx2jl1
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-unkx2jl1
  Resolved https://github.com/openai/whisper.git to commit 173ff7dd1d9fb1c4fddea0d41d704cfefeb8908c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 52.7 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803556 sha256=428755fd0472622c93a9e0ab6068231e5a563fb17b88878b82e833e81e01f4c1
  Stored in directory: /tmp/pip-ephem-wheel-cache-n2z_n_0c/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 86.5 MB/s eta 0:00:00


In [2]:
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import whisper
import torchaudio

In [3]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
# Block 4: Load and Process Audio
audio_file_path = "WTFB2.mp4"  # Set your audio file path here
audio, sample_rate = torchaudio.load(audio_file_path)

# Check if audio has more than one channel and convert to mono if needed
if audio.shape[0] > 1:
    audio = torch.mean(audio, dim=0, keepdim=True)

audio = whisper.pad_or_trim(audio.flatten()).to(DEVICE)
mel = whisper.log_mel_spectrogram(audio)

In [5]:
model = whisper.load_model("base.en")
options = whisper.DecodingOptions(language="en", without_timestamps=True)
result = model.decode(mel, options)

100%|███████████████████████████████████████| 139M/139M [00:03<00:00, 46.9MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [ ]:
# Block 6: Print Result and Save to File
print(result.text)

import os

# Get audio file name and construct output file path
audio_file_name, audio_file_ext = os.path.splitext(audio_file_path)  # Use the same variable
output_file_path = audio_file_name + "transcribed.txt"

# Save the transcription to the file
with open(output_file_path, "w") as f:
    f.write(result.text)

print(f"Transcription saved to: {output_file_path}")